# KTM Komuter Data Preprocessing

This notebook performs preprocessing steps on the KTM Komuter dataset to prepare it for our multi-objective prediction system: "Dynamic Route Scheduling & Anomaly Detection System".

## Objectives

1. Load and validate the raw data from our exploration notebook
2. Handle any missing values or inconsistencies
3. Handle outliers appropriately (flag them for anomaly detection later)
4. Encode categorical variables
5. Create a processed dataset ready for feature engineering

## Setup and Data Loading

First, let's import the necessary libraries and load our dataset.

In [1]:
# Import necessary libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

# Set visualization styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
sns.set_context("notebook", font_scale=1.2)

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 1000)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the dataset
file_path = '../data/raw/komuter_2025.csv'
df = pd.read_csv(file_path)

# Check the first few rows of the data
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (670599, 5)


,date,time,origin,destination,ridership
0,2025-01-01,00:00,Abdullah Hukum,Klang,1
1,2025-01-01,00:00,Abdullah Hukum,Telok Pulai,1
2,2025-01-01,00:00,Bangi,Batu Caves,1
3,2025-01-01,00:00,Bank Negara,Sungai Gadut,1
4,2025-01-01,00:00,Batu Tiga,Kampung Raja Uda,1


## Data Type Conversion

Let's convert date and time columns to proper datetime format.

In [ ]:
# Convert date and time columns to datetime
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df['date'] = pd.to_datetime(df['date'])

# Extract useful time components
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['day_of_week'] = df['datetime'].dt.dayofweek  # Monday=0, Sunday=6
df['day_name'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour

# Create a route column (origin-destination pair)
df['route'] = df['origin'] + ' → ' + df['destination']

# Display the updated dataframe
df.head()

## Check for Missing Values and Data Integrity

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

In [ ]:
# Check for 'Unknown' values in origin and destination columns
print("Occurrences of 'Unknown' in origin:", (df['origin'] == 'Unknown').sum())
print("Occurrences of 'Unknown' in destination:", (df['destination'] == 'Unknown').sum())

# Check for unusual or invalid values in ridership
print(f"Minimum ridership value: {df['ridership'].min()}")
print(f"Maximum ridership value: {df['ridership'].max()}")
print(f"Number of zero ridership entries: {(df['ridership'] == 0).sum()}")

## Handling Unknown Values

Let's decide how to handle the 'Unknown' values in our dataset. For our prediction system, keeping them might actually be useful since they represent real patterns in the data.

In [ ]:
# Let's analyze 'Unknown' entries to see if they follow specific patterns
unknown_origins = df[df['origin'] == 'Unknown'].copy()
unknown_destinations = df[df['destination'] == 'Unknown'].copy()

print(f"Number of 'Unknown' origins: {len(unknown_origins)}")
print(f"Number of 'Unknown' destinations: {len(unknown_destinations)}")

# Check distribution of unknown origins by hour
plt.figure(figsize=(12, 5))
sns.countplot(x='hour', data=unknown_origins)
plt.title('Distribution of Unknown Origins by Hour')
plt.tight_layout()
plt.show()

# For our prediction system, we'll keep 'Unknown' values since they represent a real pattern

## Outlier Detection and Handling

Since we're building a system that includes anomaly detection, we'll identify outliers but preserve them in a separate flag column rather than removing them.

In [ ]:
# Calculate statistical boundaries for outliers using IQR method
Q1 = df['ridership'].quantile(0.25)
Q3 = df['ridership'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Lower bound for ridership: {lower_bound}")
print(f"Upper bound for ridership: {upper_bound}")

# Create a column to flag potential outliers (for use in anomaly detection later)
df['is_statistical_outlier'] = ((df['ridership'] < lower_bound) | (df['ridership'] > upper_bound))

# Calculate how many outliers were detected
outlier_count = df['is_statistical_outlier'].sum()
print(f"Number of statistical outliers detected: {outlier_count} ({outlier_count/len(df)*100:.2f}%)")

# Display some outlier examples
print("\nExamples of outlier records:")
df[df['is_statistical_outlier']].sort_values('ridership', ascending=False).head(10)

In [ ]:
# Look for route-specific outliers
# Define a function to detect route-specific outliers
def detect_route_outliers(group):
    route_mean = group['ridership'].mean()
    route_std = group['ridership'].std()
    # Flag as outlier if more than 3 standard deviations from the route mean
    if route_std > 0:  # Protect against routes with 0 std dev
        group['is_route_outlier'] = abs(group['ridership'] - route_mean) > (3 * route_std)
    else:
        group['is_route_outlier'] = False
    return group

# Apply the function by route
df = df.groupby('route').apply(detect_route_outliers)

# Count route-specific outliers
route_outlier_count = df['is_route_outlier'].sum()
print(f"Number of route-specific outliers: {route_outlier_count} ({route_outlier_count/len(df)*100:.2f}%)")

## Feature Transformation

Let's create additional transformed features that will be useful for our prediction system.

In [ ]:
# Create binary features for time segments and days
df['is_weekend'] = df['day_of_week'] >= 5  # 5=Saturday, 6=Sunday
df['is_peak_morning'] = ((df['hour'] >= 6) & (df['hour'] <= 9) & ~df['is_weekend'])
df['is_peak_evening'] = ((df['hour'] >= 17) & (df['hour'] <= 20) & ~df['is_weekend'])
df['is_business_hours'] = ((df['hour'] >= 9) & (df['hour'] <= 17) & ~df['is_weekend'])
df['is_night_hours'] = ((df['hour'] >= 21) | (df['hour'] <= 5))

# Display the new features
df[['date', 'time', 'day_name', 'hour', 'is_weekend', 'is_peak_morning', 'is_peak_evening', 'is_business_hours', 'is_night_hours']].head(10)

## Categorical Variable Encoding

For stations and routes, we'll create frequency-based encodings to capture their relative importance.

In [ ]:
# Create frequency encodings for origin and destination stations
origin_freq = df.groupby('origin')['ridership'].count() / len(df)
dest_freq = df.groupby('destination')['ridership'].count() / len(df)

# Create dictionaries for mapping
origin_freq_map = origin_freq.to_dict()
dest_freq_map = dest_freq.to_dict()

# Apply the mappings
df['origin_freq'] = df['origin'].map(origin_freq_map)
df['destination_freq'] = df['destination'].map(dest_freq_map)

# Create popularity features based on total ridership
origin_pop = df.groupby('origin')['ridership'].sum()
dest_pop = df.groupby('destination')['ridership'].sum()

# Normalize to 0-1 range
origin_pop = (origin_pop - origin_pop.min()) / (origin_pop.max() - origin_pop.min())
dest_pop = (dest_pop - dest_pop.min()) / (dest_pop.max() - dest_pop.min())

# Create dictionaries for mapping
origin_pop_map = origin_pop.to_dict()
dest_pop_map = dest_pop.to_dict()

# Apply the mappings
df['origin_popularity'] = df['origin'].map(origin_pop_map)
df['destination_popularity'] = df['destination'].map(dest_pop_map)

# Display the new features
df[['origin', 'destination', 'origin_freq', 'destination_freq', 'origin_popularity', 'destination_popularity']].head(10)

In [ ]:
# Create cyclical encoding for hour and day of week to capture their cyclical nature
df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

# Display the cyclical encodings
df[['hour', 'day_of_week', 'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos']].head(10)

## Prepare Dataset for Route-Specific Analysis

Let's aggregate the data by route and hour to create a dataset that's better suited for our dynamic route scheduling system.

In [ ]:
# Create an aggregated dataset by route and hour
route_hour_df = df.groupby(['route', 'date', 'hour']).agg(
    total_ridership=('ridership', 'sum'),
    avg_ridership=('ridership', 'mean'),
    max_ridership=('ridership', 'max'),
    day_of_week=('day_of_week', 'first'),
    is_weekend=('is_weekend', 'first'),
    origin=('origin', 'first'),
    destination=('destination', 'first'),
    origin_popularity=('origin_popularity', 'first'),
    destination_popularity=('destination_popularity', 'first')
).reset_index()

# Add the cyclical features to the aggregated dataset
route_hour_df['hour_sin'] = np.sin(2 * np.pi * route_hour_df['hour']/24)
route_hour_df['hour_cos'] = np.cos(2 * np.pi * route_hour_df['hour']/24)
route_hour_df['day_of_week_sin'] = np.sin(2 * np.pi * route_hour_df['day_of_week']/7)
route_hour_df['day_of_week_cos'] = np.cos(2 * np.pi * route_hour_df['day_of_week']/7)

# Display the aggregated dataset
print(f"Aggregated dataset shape: {route_hour_df.shape}")
route_hour_df.head()

## Create Time Series Features

For time series forecasting, we'll create lagged features and rolling statistics.